In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes

Solving environment: / 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::certifi-2019.9.11-py37_0
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0
  - anaconda::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Solving 

In [388]:
# Import Pandas and Numpy
import pandas as pd
import numpy as np
import requests

#import dotenv for managing secrets
import os
from dotenv import load_dotenv

# Import Geocoding
from geopy.geocoders import Nominatim

# Import Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Import Folium for Visualisation
import folium # map rendering library

In [389]:
load_dotenv("config.txt")
fs_version = os.environ.get("VERSION")
fs_clientid = os.environ.get("CLIENT_ID")
fs_secret = os.environ.get("CLIENT_SECRET")
fs_limit = 30

In [390]:
# Load the dataset we saved previously in part 2
df = pd.read_csv("part2.csv")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [391]:
# Get the Geocode Address of Toronto, Canada
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print("Toronto: Lat {} Lng {}".format(lat,lng))

Toronto: Lat 43.653963 Lng -79.387207


In [392]:
# Create Map of Toronto
toronto_map = folium.Map(location=[lat,lng], zoom_start=10)

# Add Markers for all the neighbourhood data
for lat,lng,neighborhood, borough in zip(df["Latitude"],df["Longitude"],df["Neighborhood"], df["Borough"]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="blue",
        fill_color="blue",
        fill=True,
        fill_opacity=0.5
    ).add_to(toronto_map)

toronto_map

In [393]:
# Only work with Toronto Boroughs
df_central = df[df["Borough"].str.contains("Toronto")]
df_central
df_central.shape


(39, 5)

In [394]:
map_central = folium.Map([lat,lng], zoom_start=10)

for lat,lng,neighborhood,borough in zip(df_central["Latitude"], df_central["Longitude"], df_central["Neighborhood"], df_central["Borough"]):
    label = "{}, {}".format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="yellow",
        fill=True,
        fill_color="black",
        fill_opacity=0.5,
    ).add_to(map_central)

map_central

In [395]:
# Explore the nearby venues for each of out neighbourhoods
# Lets see how many neighbourhoods are in our reduced dataframe
df_central.shape

(39, 5)

In [396]:
df_central["Borough"].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [397]:
df_central["Neighborhood"].value_counts()

Stn A PO Boxes 25 The Esplanade                                                                         1
Studio District                                                                                         1
Moore Park,Summerhill East                                                                              1
Lawrence Park                                                                                           1
Business Reply Mail Processing Centre 969 Eastern                                                       1
Harbourfront                                                                                            1
The Danforth West,Riverdale                                                                             1
Roselawn                                                                                                1
Brockton,Exhibition Place,Parkdale Village                                                              1
Christie                                      

In [398]:
def get_nearby(names, lats, lngs, radius=500, limit=fs_limit):
    
    # Set Foursquare Authentication Credentials
    tkn = "client_id={}&client_secret={}&v={}".format(fs_clientid, fs_secret, fs_version)
    venue_list = []
    
    for name, lat, lng in zip(names, lats, lngs):
        # Set the API call
        api = "https://api.foursquare.com/v2/venues/explore?ll={},{}&radius={}&limit={}&{}".format(lat,lng,radius,limit,tkn)
        print("Getting Data for {}".format(name))
        
        # Make the API call
        result = requests.get(api).json()
        
        # Check some valid data was returned
        if len(result["response"]["groups"]) > 0:
            venues = result["response"]["groups"][0]["items"]

            for venue in venues:
                venue_list.append([(name, lat, lng, 
                                  venue["venue"]["name"], 
                                  venue["venue"]["location"]["lat"],
                                  venue["venue"]["location"]["lng"],
                                  venue["venue"]["categories"][0]["name"])])
            
        else:
            print("No results returned for {}".format(name))
    
    nearby = pd.DataFrame([item for row in venue_list for item in row])
    nearby.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    return nearby


In [399]:
df_near = get_nearby(df_central["Neighborhood"], df_central["Latitude"], df_central["Longitude"])
df_near

Getting Data for The Beaches
Getting Data for The Danforth West,Riverdale
Getting Data for The Beaches West,India Bazaar
Getting Data for Studio District
Getting Data for Lawrence Park
Getting Data for Davisville North
Getting Data for North Toronto West
Getting Data for Davisville
Getting Data for Moore Park,Summerhill East
Getting Data for Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Getting Data for Rosedale
Getting Data for Cabbagetown,St. James Town
Getting Data for Church and Wellesley
Getting Data for Harbourfront
Getting Data for Ryerson,Garden District
Getting Data for St. James Town
Getting Data for Berczy Park
Getting Data for Central Bay Street
Getting Data for Adelaide,King,Richmond
Getting Data for Harbourfront East,Toronto Islands,Union Station
Getting Data for Design Exchange,Toronto Dominion Centre
Getting Data for Commerce Court,Victoria Hotel
Getting Data for Roselawn
Getting Data for Forest Hill North,Forest Hill West
Getting Data for The Annex,Nort

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
...,...,...,...,...,...,...,...
855,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ashbridge Estate,43.664691,-79.321805,Garden
856,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
857,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
858,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher


In [400]:
# Check how many rows were fetched
df_near.shape

(860, 7)

In [401]:
# Examine the count of nearby venues per neighbourhood
df_near.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown,Grange Park,Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16


In [402]:
# Check how many category types were returned
df_near["Venue Category"].value_counts()

Coffee Shop              64
Café                     60
Park                     29
Italian Restaurant       25
Restaurant               24
                         ..
College Arts Building     1
Food Court                1
Poke Place                1
Falafel Restaurant        1
Dog Run                   1
Name: Venue Category, Length: 192, dtype: int64

In [403]:
df_near.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [404]:
# Analyse the neighbourhoods
print(df_near.shape)

toronto_onehot = pd.get_dummies(df_near[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.shape)


(860, 7)
(860, 192)


In [405]:
# Add in the Neighborhood Column
toronto_onehot["Neighborhood"] = df_near["Neighborhood"]

#Reorder the Data Frame Columns to make Neighborhood the first
# Build a column list in the order
fixedcols = ["Neighborhood"]
othercols = toronto_onehot.columns.to_list()
othercols.remove("Neighborhood")
fixedcols = fixedcols + othercols

#Use the column list as a selection criteria
toronto_onehot=toronto_onehot[fixedcols]
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [406]:
# Get the Top 10 venues for each neighbourhood
grp = toronto_onehot.groupby("Neighborhood").mean().reset_index()
grp

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.058824,0.058824,0.058824,0.058824,0.176471,0.117647,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000


In [407]:
# Helper Function to get the most common N venue types for a neighborhood
# Transposes the data, sorts and extracts the head topN
# These are returned as a list
def get_top_venues(mean_grouped_neighborhoods, neighborhood, topN=10):
    
    temp = mean_grouped_neighborhoods[mean_grouped_neighborhoods["Neighborhood"]==neighborhood].T.reset_index()
    temp.columns = ["feature", "frequency"]
    temp = temp.iloc[1:]
    temp = temp.sort_values("frequency", ascending=False)
    temp = temp.head(topN)
    topN_list= [neighborhood]
    topN_list = topN_list + temp["feature"].to_list()
    return topN_list

       

In [418]:
# Set the Top N most common features to extract for each neighborhood
topN=10

# Dynamically build the columsn set for the specified features
cols1 = ["Neighborhood"]
cols2 = ["Feature" + str(i) for i in range(topN)]
cols=cols1 + cols2

# Create an empty dataframe with the required columns
df_topN = pd.DataFrame(columns = cols)

# For each row in our group get the neighbourhood and extract
# the top N most common venue categories for that neighnoorhood
# Append the results to a dataframe df_topN
for i in np.arange(grp.shape[0]):
    neighborhood = grp.iloc[i,0]
    topN_list = get_top_venues(grp,neighborhood,topN) 
    df_topN.loc[i]=topN_list


In [425]:
# Apply Clustering Algorithms
kclusters = 4

# Drop the Neighbourhood Column in a new dataframe
toronto_grp_cluster = grp.drop("Neighborhood", axis = 1)
toronto_grp_cluster

# Run k-means clustering algorithm
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grp_cluster)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:39] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [426]:
df_topN["Cluster Labels"] =  kmeans.labels_
df_topN["Cluster Labels"].value_counts()

0    34
2     3
3     1
1     1
Name: Cluster Labels, dtype: int64

In [427]:
#Merge the two datasets together to create a dataset of everything we have analysed so far
df_merged = pd.merge(df, df_topN, on="Neighborhood", how="inner")
df_merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Pub,Trail,New American Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Pizza Place,Bubble Tea Shop,Spa,Caribbean Restaurant,Restaurant,Coffee Shop,0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,Park,Sandwich Place,Burrito Place,Liquor Store,Movie Theater,Steakhouse,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Italian Restaurant,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Brewery,Comfort Food Restaurant,Seafood Restaurant,Gay Bar,Fish Market,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Swim School,Park,Bus Line,Airport,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,2


In [428]:
# create map using the Toronto LatLng derived earlier
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(df_merged["Latitude"], df_merged["Longitude"], df_merged["Neighborhood"], df_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + " Cluster " + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [429]:
# Look at Cluster 0
df_merged[df_merged["Cluster Labels"]==0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Pub,Trail,New American Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Pizza Place,Bubble Tea Shop,Spa,Caribbean Restaurant,Restaurant,Coffee Shop,0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,Park,Sandwich Place,Burrito Place,Liquor Store,Movie Theater,Steakhouse,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Italian Restaurant,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Brewery,Comfort Food Restaurant,Seafood Restaurant,Gay Bar,Fish Market,0
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Food & Drink Shop,Gym,Hotel,Clothing Store,Park,Breakfast Spot,Sandwich Place,Airport,New American Restaurant,Middle Eastern Restaurant,0
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,Sporting Goods Shop,Coffee Shop,Clothing Store,Metro Station,Cosmetics Shop,Dessert Shop,Diner,Rental Car Location,Restaurant,Park,0
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,Dessert Shop,Café,Pizza Place,Sandwich Place,Gym,Sushi Restaurant,Coffee Shop,Italian Restaurant,Park,Seafood Restaurant,0
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,Pub,Coffee Shop,Bagel Shop,Restaurant,Sports Bar,Light Rail Station,Supermarket,Sushi Restaurant,Fried Chicken Joint,Liquor Store,0
11,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,Café,Italian Restaurant,Restaurant,Bakery,Playground,Bank,Butcher,Sandwich Place,Market,Coffee Shop,0
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,Gay Bar,Burger Joint,Pub,Beer Bar,Gastropub,Bookstore,Breakfast Spot,Pizza Place,Mexican Restaurant,Salon / Barbershop,0


In [430]:
# Look at Cluster 1
df_merged[df_merged["Cluster Labels"]==1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Cluster Labels
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.38316,Summer Camp,Tennis Court,Airport,Museum,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,1


In [431]:
# Look at Cluster 2 (Playgroudns and Parks)
df_merged[df_merged["Cluster Labels"]==2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Cluster Labels
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Swim School,Park,Bus Line,Airport,Music Venue,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,2
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,Park,Playground,Trail,New American Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,2
23,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,Sushi Restaurant,Park,Trail,Jewelry Store,Airport,Museum,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,2
